In [27]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.5.0


## データの読み込み

In [28]:
# MNIST データセットを読み込む
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Softmax 回帰によるモデルを作成

softmax(多項ロジスティック)関数は、多変量のロジスティック回帰を利用した多値分類のアルゴリズム。


<img src="./img/softmax.png" style="height: 40px">


Softmax 回帰では、x の重み合計を計算し、バイアスを加え、そして softmax を適用する。
以下のようなイメージ。

* y が目的変数 (0-9の数字)
* x が説明変数 (256 階調のグレースケール、各ピクセルの濃淡を 0.0-1.0 で表したもの)
* W が重み
* b がバイアスの行列

<img src="./img/softmax1.png" style="height: 130px">

式で表すと、

<img src="./img/softmax2.png" style="height: 100px">


今回は、最適な重みとバイアスを推定するための誤差関数（損失関数）として、交差エントロピー (cross entropy) と呼ばれる指標を用います。この交差エントロピーが最も小さくなるように、TensorFlow に `GradientDescentOptimizer` として実装されている、勾配降下法 ( `Gradient descent` ) を用いて学習を進めます。

In [30]:
# 入力データ格納用の 784px 分のプレースホルダを作成
x = tf.placeholder(tf.float32, [None, 784])

# 重み (784 ✕ 10 の行列) の Variable を定義
W = tf.Variable(tf.zeros([784, 10]))

# バイアス (長さ 10 の行列) の Variable を定義
b = tf.Variable(tf.zeros([10]))

# ソフトマックス回帰による予測式を定義
y = tf.nn.softmax(tf.matmul(x, W) + b)

# 出力データ (予測値) 格納用のプレースホルダ
y_ = tf.placeholder(tf.float32, [None, 10])

# 交差エントロピーが最も小さくなるように、学習を行う式を定義
loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 実際の値と予測された値が同じであるか確認
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

# 平均値を求め、予測精度を求める
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 初期化
tf.global_variables_initializer()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
        
# 学習を 1000 回繰り返す
for i in range(1000):
    # 訓練用データから 100 件取得
    batch_xs, batch_ys = mnist.train.next_batch(100)
    # train_spep を実行
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val))
    

Step: 0, Loss: 2.119464, Accuracy: 0.194800
Step: 100, Loss: 0.384369, Accuracy: 0.893200
Step: 200, Loss: 0.337418, Accuracy: 0.906300
Step: 300, Loss: 0.315937, Accuracy: 0.912500
Step: 400, Loss: 0.313405, Accuracy: 0.911900
Step: 500, Loss: 0.308517, Accuracy: 0.915000
Step: 600, Loss: 0.299391, Accuracy: 0.912800
Step: 700, Loss: 0.297278, Accuracy: 0.913800
Step: 800, Loss: 0.294411, Accuracy: 0.916700
Step: 900, Loss: 0.288827, Accuracy: 0.917600


## モデルの評価

In [32]:
# Testing
train_loss = sess.run(loss, feed_dict={x: mnist.train.images, y_: mnist.train.labels})
train_accuracy = sess.run(accuracy, feed_dict={x: mnist.train.images, y_: mnist.train.labels})
test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})

# 学訓練エラー、訓練精度、テスト精度
std_output = 'Train Loss: %s, \t Train Accuracy: %s, \t Test Accuracy: %s'
print(std_output % (train_loss, train_accuracy, test_accuracy))


Train Loss: 0.29550225, 	 Train Accuracy: 0.91685456, 	 Test Accuracy: 0.9177


## モデルの保存

In [33]:
saver = tf.train.Saver()
save_path = saver.save(sess, "model/model.ckpt")
print("Model saved in path: %s" % save_path)


Model saved in path: model/model.ckpt
